# Text Mining com Python e Whoosh: Cartas Brasileiras (PHPB-Ba)

### Valter Moreno

**Fundação Getúlio Vargas (FGV)**<br/>
**Mestrado em Matemática Aplicada (EMAp)**<br/>
**Sistemas de Recuperação de Informação - 2019**<br/>

## Objetivos do projeto

Neste projeto, vamos analisar os manuscritos modernizados do Corpus Eletrônico de Documentos Históricos do Sertão [CE-DOHS](http://www.tycho.iel.unicamp.br/cedohs/corpora.html).

## Corpus analisado

O Corpus Compartilhado Diacrônico – Cartas Brasileiras (PHPB-Ba) é disponibilizado no website do projeto Plataforma Corpus Eletrônico de Documentos Históricos do Sertão [CE-DOHS]. O projeto é coordenado por Zenaide de Oliveira Novais Carneiro (UEFS/Fapesb/CNPq) e Mariana Fagundes de Oliveira Lacerda (UEFS/Fapesb).

O corpus analisado neste projeto consiste de 1.185 cartas com datas no período de 1823 a 2000. As cartas provêm de 14 corporas distintos, listados no [website do CE-DOHS](http://www.tycho.iel.unicamp.br/cedohs/corpora.html).

## Análise

### Importação de bibliotecas

In [1]:
import os.path
import pandas as pd
import numpy as np
import glob

import re

import string
from string import punctuation

from collections import defaultdict

from spellchecker import SpellChecker

import spacy
nlp = spacy.load("pt_core_news_sm")

from gensim import corpora
from gensim import models

import nltk
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import PortugueseStemmer

from whoosh.index import create_in, open_dir
from whoosh.fields import *
from whoosh.analysis import RegexTokenizer, LanguageAnalyzer, StopFilter, LowercaseFilter, StemFilter
from whoosh.qparser import QueryParser
from whoosh.query import Phrase

pd.set_option('display.max_colwidth', -1) # Customização do display de dataframes para que
                                          # todo o conteúdo das colunas seja mostrado

### Importação dos textos do corpus

O corpus de 1.185 cartas foi carregado num vetor do R. O conteúdo do primeiro arquivo carregado é mostrado abaixo.


In [2]:
caminho = "CE-DOHS/*.txt"
arquivos  = glob.glob(caminho)
print('Há um total de', len(arquivos), 'no diretório do corpus.', '\n')
print('Nomes dos seis primeiros arquivos:')
for i in range(0,6):
    print(arquivos[i])

Há um total de 2370 no diretório do corpus. 

Nomes dos seis primeiros arquivos:
CE-DOHS\01-AAD-17-04-1962-mod.txt
CE-DOHS\01-AAD-17-04-1962-orig.txt
CE-DOHS\01-ACST-14-01-1952-mod.txt
CE-DOHS\01-ACST-14-01-1952-orig.txt
CE-DOHS\01-AFS-28-04-1956-mod.txt
CE-DOHS\01-AFS-28-04-1956-orig.txt


Como pode ser visto acima, os arquivos baixados do CE-DOHS incluem as cartas originais e as cartas com o português modernizado. Em nossa análise, usaremos apenas as cartas com o português modernizado.

### Extração de metadados e texto dos arquivos

Os arquivos do CE-DOHS incluem metadados em seu cabeçalho. Identificamos os seguintes campos:

  * Identificação do corpus
  * Nome da pesqisadora  
  * Identificação da carta
  * Nome do autor da carta
  * Nome do destinatário
  * Data da carta
  * Identificação de carta modernizada
  * Identificação da codificação (UTF-8)
  * Número da carta
  * Descrição do manuscrito (ex., local de arquivo, tipo de papel)
  
Criamos um dataframe com os metadados relevantes (identificação da carta, autor, destinatário, e data), além do próprio texto do documento. O texto de cada carta foi processado da seguinte forma:

  * removemos espaços desnecessários dos nomes dos remetentes e destinatários; 
  * tranformamos as cadeias de caracter das datas originais em datas;
  * removemos o cabeçalho, com os metadados do arquivo; parte deles foi aproveitada para preencher os valores de identificação da carta, autor, destinatário, e data;
  * removemos o texto explicativo geralmente adicionado a cada carta, com informações tais como o tipo de papel usado, o número de folhas, onde a carta estava guardada, etc.
  * removemos notas inseridas no manuscrito para identificar mudanças de página, rasuras, borrados, etc.
  * removemos notas de rodapé, tipicamente explicando que anotações foram feitas no manuscrito

In [3]:
def data_id(id):
    data_num = re.compile('[0-9]{2}-[0-9]{2}-[0-9]{4}')
    data = data_num.findall(id)
    if data:
        try:
            return pd.to_datetime(data[0], format='%d-%m-%Y')
        except:
            return np.NaN
    else:
        return np.NaN

cartas = pd.DataFrame(columns=['id', 'de', 'para', 'data', 'texto'])

meses = ('janeiro','fevereiro','mar','abril','maio','junho', 'julho','agosto','setembro','outubro','novembro','dezembro')

for root, dirs, files in os.walk(r'CE-DOHS'):
    if files:
        for fileid in files:
            
            if not fileid.endswith('-mod.txt'):
                continue
            
            with open(os.path.join(root, fileid), mode="r", encoding="utf-8") as doc:
                id = fileid
                
                texto = doc.read()
                
                linhas = texto.splitlines()
                
                de = [autor.replace(u'|:| Autor:', '').strip() \
                         for autor in linhas \
                         if u'|:| Autor:' in autor][0]
                de = " ".join(de.split())
                                
                para = [destino.replace(u'|:| Destinatário:', '').strip() \
                         for destino in linhas \
                         if u'|:| Destinatário:' in destino][0]
                para = " ".join(para.split())
                
                data = [dt.replace(u'|:| Data:', '').strip(' .').lower() \
                         for dt in linhas \
                         if u'|:| Data:' in dt][0]
                
                mes = [mes for mes in meses if mes in data]
                               
                if mes:
                    data = data.replace(mes[0], '/' + str(meses.index(mes[0]) + 1) + '/')
                    
                data_num = re.compile('[0-9\/]')
                data = data_num.findall(data)
                data = ''.join(data)
                
                try:
                    data = pd.to_datetime(data, infer_datetime_format=True)
                    if pd.isna(data):
                        data = data_id(id)
                except:
                    data = data_id(id)
                                                
                texto = linhas[15:len(linhas)]
                texto = '\n'.join(texto)
                texto = texto.replace('\n\n[pag]\n\n', '')
                texto = re.sub('\[footer:([\\n a-zA-Z0-9áàâãéèêíïóôõöúçñÁÀÂÃÉÈÍÏÓÔÕÖÚÇÑ.#\"‘’“”]*)\]',
                                 '', texto)
                texto = re.sub('\[header:[ ]*\]', '\n', texto)
                texto = re.sub('[|]*[ ]*[Ff]l. [0-9] [rv]', ' ', texto)
                texto = re.sub('[0-9][ ]*Rasura[do]*[ .]*', ' ', texto)
                texto = re.sub('[0-9][ ]*Borrado[ .]*', ' ', texto)
                texto = re.sub('\[[ ]*[Rr]ubrica[ ]*\]', ' ', texto)
                texto = re.sub('\[[ ]*[Rr]asura[ ]*\]', ' ', texto)
                texto = re.sub('\[[ ]*[Ii]nint[ ]*.[ ]*\]', ' ', texto)
                texto = texto.strip()
                
                new_row = {'id': id, 'de': de, 'para': para, 'data': data, 'texto': texto}
                cartas = cartas.append(new_row, ignore_index=True)

Seguem as cinco primeiras linhas do dataframe criado, com as cinco primeiras cartas processadas.

In [4]:
cartas.head(5)

,id,de,para,data,texto
0,01-AAD-17-04-1962-mod.txt,Arnaldo Andrade Dias (apócrifo),Maria de Lourdes de Lima Oliveira,1962-04-17,"São Paulo 17 de abril 1962\n\n Inesquicível Lurdes\n\n Adeus ao longe ,\n\n Em uma tarde tristonho , cheio de saudade , que peguei em minha rude caneta para te escrever dando as minhas notícias , que 1 de saúde vou bem só o que me mata 2 é a cruel se 3 lembrança de você .\n\n Talvez você não acredite meu amor mais quando chega 4 o domingo fico na impressão que a tarde vou te ver e a inda mais aumenta o meu sofrer pois já estou arrependido da hora que me ausentei de lá . Embora já sabia que não suportava viver ausente de te meu amor , tentei só para sofrer , parece-me que tu és a única pessoa que Deus deixou para eu amar com todo amor que meu jovem coração encerra .\n\n\n\n Sem mais abraços fortes para você e aceite mil recordações .\n\n deste que te ama sem falsidade até o fim , do seu amor que muito te quer .\n\n Arnaldo Andrade Dias"
1,01-ACST-14-01-1952-mod.txt,Antonio Carneiro da Silva Tuy,Maria de Souza Estrela,1952-01-14,"Fazenda Bom Jardim 14 – 1 – 52 .\n\n Minha querida noiva Maria . Enviu-o os mesmos ... “Beijinhos.” Faço votos sinceros ao Divino Criador , que paz seja sempre o teu amparo.” E aos teus , que para mim será grande reforma de satisfação . Se com os méritos de Deus ; acolhermos singular transformação e vigor . 1 Eu e todos vamos na forma do costume . Em segundo lugar te falo que fomos bem de viagem . Tudo mais tu me relata , e[m] observação . Nada de abuso . O mais , o próprio tempo está incumbido para tudo . Sim , tu não notou quanto nós te abusamos ? Te deixamos muito abatida com os incômodos . Já quer que eu volte sexta-feira . Espere , vou empregar meios de ir . Sonhei contigo esta noite . Te abraçando e te beijando , dentro de uma casa , que muito silenciosa estava ; parecia ser nossa . A ti meus agradecimentos , votos e cuidados . Estou grato pelo presente , muito obrigado Segue estas bananinhas para tu , e minha mãe velha , para merendarem . E 20 litros de milho . Evandro envia-te as mesmas , e o velho . Um abraço em nossa mãe . Aceite minhas saudosas lembranças . A.C.S.T .\n\n Antonio Carneiro da Silva . 2\n\n <minha pena está – 2 – [ . ] – escarranchada.> 3\n\n 1 Há pontos aleatórios após a palavra .\n\n 2 Escrito na margem inferior .\n\n 3 Escrito verticalmente , de baixo para cima ."
2,01-AFS-28-04-1956-mod.txt,Antonio Fortunato da Silva.,João Carneiro de Oliveira.,1956-04-28,"Carrancudo 28 de Abril de 1956 Prezado queridinho estimado amigo Pitanga esta duas linha lhe dizer procura a notícia de você e todo seu todo meu vai como Deus quiser como vai o nosso inverno a qui faz muito sro nada fazer não patenmos 1 sem queridinho . p. compadres Pedro já esqueceu de mim não esqueça mande dizer como vai o senhor compadre eu estimo de longe de perto não posso adeus a te , se Deus quiser nada mais prezado seu amigo 2 sim meu queridinho amigo Augusto como vai amigo você também não esqueç[.]a 3 de mim lembra do nosso [?] passado se Deus mim ordem faço intenção de ir de pura de são João Deus e saber nada mais do seu prezado amigo\n\n Antonio Fortunato Silva Agsto Augusto lembrança daquela\n\n dei menina Maria Jertrudes meu Coração [?] a tina aimario\n\n 1 Após a palavra há traço vertical cortado por dois traços horizontais .\n\n 2 Após a palavra há traço vertical cortado por um traço horizontal .\n\n 3 Desgaste do papel devido à dobra ."
3,01-APACA-14-01-1980-mod.txt,Adelmário Carneiro de Araújo,Eliana de Oliveira Lima,1980-01-14,"Valente , 14 de janeiro de 1980\n\n Olá Amor tudo bem ?\n\n Olha querida espero que esta a encontre com paz e saúde juntamente com os seus manos e pais e familiares .\n\n Aninha quando você receber esta carta me responda logo por que estou com medo de você não receber esta e pensar que eu não lhe escrevi , por que tenho dúvidas no endereço que não conheço , mas aqui em Valente tudo bem , conheço tudo .\n\n Quando você subiu no ônibus me deu uma vontade tr

### Indexação dos documentos no Whoosh

Usamos o módulo [Whoosh](https://whoosh.readthedocs.io/en/latest/index.html) para criar o índice dos documentos e adicionar o texto das cartas e os metadados. Isso é feito por meio de um esquema ([*schema*](https://whoosh.readthedocs.io/en/latest/schema.html)), que define os campos que conterão as informações sobre os documentos indexados. 

O esquema criado contém os seguintes campos (com os seus tipos entre parênteses):

  - id (ID único): nome do arquivo
  - de (TEXT): remetente da carta
  - para (TEXT): destinatário
  - data (DATETIME): data da carta
  - texto (TEXT): texto da carta pré-processado
  
Quando um documento é adicionado ao índice criado com base num esquema, um analizador (*analyzer*) é chamado para processar as informações associadas ao documento e guadar os resultados nos campos correspondentes (https://whoosh.readthedocs.io/en/latest/schema.html). Para campos do tipo TEXT, são guardadas a frequência com que termos (*tokens*) aparecem e as suas posições no texto.

No Whoosh, um analizador é geralmente composto por um tokenizador (*tokenizer*) e um ou mais filtros (*filters*) (https://whoosh.readthedocs.io/en/latest/api/analysis.html). O tokenizador quebra o texto fornecido em seus termos elementares (ex., palavras). Os filtros recebem esses termos e os transformam (ex., os convertem para minúsculas, eliminam palavras de uma lista de *stopwords*).

O analyzador padrão do Whoosh é o [StandardAnalyzer](https://whoosh.readthedocs.io/en/latest/api/analysis.html#whoosh.analysis.StandardAnalyzer), criado para processar documentos em inglês. Como os documentos do nosso corpus estão em português, foi necessário utilizar o [LanguageAnalyzer](https://whoosh.readthedocs.io/en/latest/api/analysis.html#whoosh.analysis.LanguageAnalyzer), que inclui os seguintes filtros:
  
  - *LowercaseFilter*: converte os termos para minúsculas
  - *StopFilter*: elimina termos correspondentes a palavras muito comuns e de pouco valor semântico (ex., artigos definidos e indefinidos), renumerando (ou não) a posição de ocorrência de cada termo remanescente no texto; 
  - *StemFilter*: remove sufixos de termos usando, geralmente, o algoritmo de Porter (https://whoosh.readthedocs.io/en/latest/api/analysis.html#whoosh.analysis.StemFilter)
  
Abaixo, vemos as línguas disponíveis no *LanguageAnalyzer* e verificamos se há função de *stemming* e uma lista de *stopwords* para o português:

In [5]:
from whoosh import lang
print('Línguas disponíveis:', lang.languages)
print('Stemmer em português disponível?', lang.has_stemmer('pt'))
print('Stopwords em português diponível?', lang.has_stopwords('pt'))

Línguas disponíveis: ('ar', 'da', 'nl', 'en', 'fi', 'fr', 'de', 'hu', 'it', 'no', 'pt', 'ro', 'ru', 'es', 'sv', 'tr')
Stemmer em português disponível? True
Stopwords em português diponível? True


Como queríamos realizar buscas nos campos do remetente e do destinatário, não era possível usar o *languageAnalyzer* diretamente. No caso de nomes próprios, não podemos passar os termos para minúsculas, remover *stopwords*, ou fazer o *stemming*. Dessa forma, criamos outro *analyzer* apenas com o [RegexTokenizer](https://whoosh.readthedocs.io/en/latest/api/analysis.html#whoosh.analysis.RegexTokenizer). Os resultados gerados com ele são ilustrados abaixo. 

In [6]:
RexTokenizer = RegexTokenizer(expression='[a-zA-Z0-9áàâãéèêíïóôõöúçñÁÀÂÃÉÈÍÏÓÔÕÖÚÇÑ]*')
[token.text for token in RexTokenizer(u"Benicio, Álvares. da Conceição")]

['Benicio', '', '', 'Álvares', '', '', 'da', '', 'Conceição', '']

Para fins de avaliação dos resultados das nossas buscas, criamos também outro *analyzer* apenas com tokenizador *RegexTokenizer* e os filtros *LowercaseFilter* e *StopWords*. Mostramos abaixo o resultado para o texto da primeira carta em nosso corpus.

In [7]:
BaseAnalyzer = RegexTokenizer() | LowercaseFilter() | StopFilter(lang='pt')

lista = [token.text for token in BaseAnalyzer(cartas.texto[0])]
sorted(lista)

['17',
 '1962',
 'abraços',
 'abril',
 'aceite',
 'acredite',
 'adeus',
 'ama',
 'amar',
 'amor',
 'amor',
 'amor',
 'amor',
 'andrade',
 'arnaldo',
 'arrependido',
 'aumenta',
 'ausente',
 'ausentei',
 'bem',
 'caneta',
 'chega',
 'cheio',
 'coração',
 'cruel',
 'dando',
 'deixou',
 'deste',
 'deus',
 'dias',
 'domingo',
 'embora',
 'encerra',
 'escrever',
 'falsidade',
 'fico',
 'fim',
 'fortes',
 'hora',
 'impressão',
 'inda',
 'inesquicível',
 'jovem',
 'lembrança',
 'longe',
 'lurdes',
 'lá',
 'mata',
 'mil',
 'notícias',
 'parece',
 'paulo',
 'peguei',
 'pessoa',
 'pois',
 'quer',
 'recordações',
 'rude',
 'sabia',
 'saudade',
 'saúde',
 'sofrer',
 'sofrer',
 'suportava',
 'talvez',
 'tarde',
 'tarde',
 'tentei',
 'todo',
 'tristonho',
 'ver',
 'viver',
 'vou',
 'vou',
 'és',
 'única']

Para fins de comparação, segue o mesmo teste para o *languageAnalyzer*.

In [8]:
LanguageAnalyzerPt = LanguageAnalyzer(lang='pt')

lista = [token.text for token in LanguageAnalyzerPt(cartas.texto[0])]
sorted(lista)

['17',
 '1962',
 'abrac',
 'abril',
 'aceit',
 'acredit',
 'adeus',
 'ama',
 'amar',
 'amor',
 'amor',
 'amor',
 'amor',
 'andrad',
 'arnald',
 'arrepend',
 'aument',
 'ausent',
 'ausent',
 'bem',
 'canet',
 'cheg',
 'chei',
 'coraçã',
 'cruel',
 'dand',
 'deix',
 'dest',
 'deus',
 'dias',
 'doming',
 'embor',
 'encerr',
 'escrev',
 'falsidad',
 'fic',
 'fim',
 'fort',
 'hor',
 'impressã',
 'inda',
 'inesquic',
 'jov',
 'lembranc',
 'long',
 'lurd',
 'lá',
 'mat',
 'mil',
 'notíc',
 'parec',
 'paul',
 'pegu',
 'pesso',
 'pois',
 'quer',
 'record',
 'rud',
 'sab',
 'saudad',
 'saúd',
 'sofr',
 'sofr',
 'suport',
 'talvez',
 'tard',
 'tard',
 'tent',
 'tod',
 'tristonh',
 'ver',
 'viv',
 'vou',
 'vou',
 'és',
 'únic']

A definição do analizador pode ser feita na própria definição do esquema. Inicialmente, criamos o esquema e o índice (a partir do esquema), e indexamos os documentos do nosso corpus usando o *analyzer* baseado apenas no *RegexTokenizer* e o *analyzer* baseado no *LanguageAnalyzer* sem o filtro de *stemming*.

In [9]:
if os.path.exists('index_cartas'):
   for file in os.listdir('index_cartas'):
        file_path = os.path.join('index_cartas', file)
        if os.path.isfile(file_path):
            os.remove(file_path)
   os.rmdir('index_cartas')

In [10]:
schema = Schema(id=ID(unique=True, stored=True), 
                de=TEXT(stored=True,
                        analyzer=RegexTokenizer(expression='[a-zA-Z0-9áàâãéèêíïóôõöúçñÁÀÂÃÉÈÍÏÓÔÕÖÚÇÑ]*')), 
                para=TEXT(stored=True,
                          analyzer=RegexTokenizer(expression='[a-zA-Z0-9áàâãéèêíïóôõöúçñÁÀÂÃÉÈÍÏÓÔÕÖÚÇÑ]*')), 
                data=DATETIME(stored=True), 
                texto=TEXT(stored=True,  
                           analyzer=RegexTokenizer() | LowercaseFilter() | StopFilter(lang='pt')))

os.mkdir('index_cartas')
ix = create_in('index_cartas', schema)

writer = ix.writer()

for i in cartas.index:
    if pd.isna(cartas.de[i]):
        cartas.de[i] = ''
        
    if pd.isna(cartas.para[i]):
        cartas.para[i] = '' 
    
    if pd.isna(cartas.data[i]):
        writer.add_document(id=cartas.id[i], 
                            de=cartas.de[i], 
                            para=cartas.para[i], 
                            texto=cartas.texto[i])
    else:
        writer.add_document(id=cartas.id[i], 
                            de=cartas.de[i], 
                            para=cartas.para[i], 
                            data=cartas.data[i], 
                            texto=cartas.texto[i])
writer.commit()

### Buscas (Questão 1 da Lista 1):

O corpus analisado inclui 43 cartas escritas por Benício Penalva, comerciante de peles de Itapicuru, para o Coronel Cícero Dantas Martins, Barão de Jeremoabo (1880-1903). As cinco primeiras são listadas a seguir, juntamete com os temas tratados em cada uma delas.

In [11]:
jeremoabo = cartas.loc[cartas.de.str.startswith('Benicio Penalva'), :].copy()
jeremoabo['tema'] = ['emprego, educação, saúde', 'política', 'emprego, política', 'emprego, política', 'outros', 
                     'outros', 'emprego', 'emprego', 'emprego, educação', 'emprego', 'emprego', 'política', 
                     'política', 'política', 'política, saúde', 'política', 'política', 'política', 'política', 
                     'política', 'política', 'política, saúde', 'política, saúde', 'política, saúde', 'política', 
                     'política', 'política', 'política', 'política', 'política', 'política, educação', 'política', 
                     'política', 'política', 'política, saúde', 'política, saúde', 'emprego, saúde', 'política', 
                     'política', 'política, saúde', 'política', 'educação, emprego', 'emprego, política']
jeremoabo.head(5)

,id,de,para,data,texto,tema
716,354-BPF-27-03-1890-mod.txt,Benicio Penalva de Faria (Benicio Penalva).,"Cícero Dantas Martins, Barão de Jeremoabo.",1890-03-27,"Meu caro Primo compadre e amigo Sr. Barão .\n\n Bahia 27 de Março de 1890\n\n Aqui cheguei com minha família no dia 15 deste e só no dia 25 - pude ir a casa do Doutor Salustio , e não o encontrando , tornei hoje , e com ele estive conversando : apresentei-lhe a carta de Vossa Excelência como me ordenou . Disse-me o Doutor Salustio , que soube estarem esperando hoje pelo Visconde 5 do- Guahy , e que não sabia se era exata essa notícia : preveni ao- Doutor Salustio para falar ao Guahy , sobre um emprego no novo hospital da misericórdia , segundo Vossa Excelência me ter dito que queria ver se podia obter com o Guahy um bom emprego para mim-\n\n no dito hospital . Vossa Excelência sabe que muito e muito preciso , e que só me arrisquei a vir com a família para esta capital , com único fim de dar a meus filhos uma educação mais ou menos regular , - pois é o meu maior desideratum . Já matriculei dois meninos- no Lyceo , e minha filha hoje , foi 6 também matriculada no externato normal de Senhoras . Se eu puder conseguir os meus- desejos , é um meio de vida que dou a ela , visto não termos outro de que possa ela viver . Sua afilhada , e o do Doutor Severino , ainda não pôde levá-los a escola , por faltar- me um\n\n atestado médico , de como não sofrem moléstia alguma contagiosa , do 7 que só poderei dar andamento depois da semana Santa . Tenho andado tanto aqui , que já 8 me doem as pernas , de maneiras que , para caminhar , é em marcha muito lenta . Tenho feito muita despesa com as arrumações dos meninos , bem entendido , de conformidade a minhas forças , por que cada um vive como pode , e- como sabe , não tenho recursos . Provisoriamente estou morando- na rua do Alvo , nº 201 – 2º andar , em quanto procuro uma roça para ir tendo alguma cousa com que possa ir dando a sub-\n\n a subsistência a meus filhos : Deus é grande e não desampara- a ninguém , principalmente a quem faz diligência com fé nele . Logo que ache a roça e arrende , preciso comprar uns burros 9 , e- desde já lhe previno , por serem os de Vossa Excelência reforçados . Meu filho Francisco , desde que aqui- chegou , entrou em tratamento com o Doutor Ribeiro dos Santos , por se achar sofrendo bastante dos olhos . Aqui tem chovido alguma cousa , e consta-me que por lá também ; Deus permita que continue . Sua Comadre está muito satisfeita aqui , - por ter o mesmo desejo que eu\n\n tenho de ver os filhos educados . Não temos tido alteração em nossa saúde graças a Deus . Muito estimo que ao lado da Excelentíssima Baronesa frua perfeita saúde e todos os- bens . Abençõe sua afilhada , e disponha sempre com a maior- franqueza , do pouco préstimo deste que com muita estima e- melhor consideração é\n\n De Vossa Excelência Primo compadre e amigo pelo Coração\n\n Benicio Penalva . P. S. Recebeu uma carta com a- escritura para o Sr. Boaventura ? Remetia pelo correio do Timbó- que devia seguir no dia 17 deste .\n\n O mesmo","emprego, educação, saúde"
717,355-BPF-13-08-1890-mod.txt,Benicio Penalva de Faria (Benicio Penalva).,"Cícero Dantas Martins, Barão de Jeremoabo.",1890-08-13,"Meu Prezado Primo compadre e amigo Sr. Barão\n\n Bahia 13 de Agosto de 1890\n\n Anteontem estive com Joãozinho , por tê-lo encontrado no comércio , e- perguntando-lhe por Vossa Excelência , disse-me estar incomodado de sua saúde , o- que muito sinto : desejando que ao receber esta , se ache vigoroso ao lado da Excelentíssima Baronesa , com quem receberá nossas felicitações . O que há de mais importante aqui é a grande questão de esgotos 10 que não lhe é mais estranho . O marechal Hermes está mal de saúde , ouvi dizer que os- médicos o privarão de falar e que de vez em quando apresenta-lhe uma- hemorragia de sangue . O < redator > 11 do pequeno jornal tem brilhado\n\n nessa grande questão de esgotos , ultimamente disse que se o governador não sabia gover

Em 33 das cartas, Benício trata de disputas políticas e de ameaças e traições de pessoas ligadas ao Barão de Jeremoabo. Há inclusive menções a atentados e assassinatos relaciondados a essas disputas. Em 11, solicita colocações para ele mesmo ou para seu filho Francisco; em nove, trata de problemas de saúde seus e de sua família; em quatro, da educação de seus filhos. 

Utilizamos esse subjconto de documentos para testar e avaliar as funcionalidades de busca oferecidas pelo Whoosh.

Primeiro, criamos um objeto de busca (*searcher*) para o índice gerado anteriormente. O objeto de busca possui métodos úteis para obter informações do índice, como *lexicon* (https://whoosh.readthedocs.io/en/latest/searching.html), que retorna as cadeias de caracter (termos ou *tokens*) geradas para um dado campo. Mostramos abaixo exemplos de tokens gerados para os campos *texto* e *de*.

In [12]:
searcher = ix.searcher()
termos = [x.decode('utf-8') for x in list(searcher.lexicon("texto"))]
nomes =  [x.decode('utf-8') for x in list(searcher.lexicon("de"))]

In [13]:
print('Total de termos para o campo "texto":', len(termos))
print(termos[0:len(termos):500])

Total de termos para o campo "texto": 22328
['00', '43', 'acharei', 'agradeçamos', 'ana', 'apt', 'atrasamos', 'beneficiar', 'caixaqui', 'caótico', 'colchão', 'confianças', 'conversar', 'cânticos', 'deram', 'desvaneceram', 'dobrado', 'encarniçado', 'escondidas', 'exames', 'faças', 'fra', 'goze', 'ildefonso', 'influem', 'itinerário', 'legoas', 'machos', 'melhorando', 'mpromissos', 'novilha', 'ouvidos', 'pena', 'podera', 'pretencioso', 'pródigo', 'receberemos', 'reporte', 'romaria', 'seite', 'sonhos', 'telefonar', 'trambalhões', 'vaqueijada', 'voce']


In [14]:
print('Total de termos para o campo "de":', len(nomes))
print(nomes[0:len(nomes):50])

Total de termos para o campo "de": 678
['', 'Angelica', 'Belisario', 'Clarival', 'Dr', 'Fortunato', 'Inácio', 'Leal', 'Mariana', 'Nolasco', 'Pitanga', 'Salvador', 'Telles', 'Zulmira']


Em seguida, criamos um objeto do tipo *QueryParser*, informando o campo padrão que será o objeto das buscas e o esquema do índice em as buscas serão feitas. O próximo passo é chamar o método *parse* com a busca representada como uma cadeia de caracteres (https://whoosh.readthedocs.io/en/latest/querylang.html).

In [15]:
busca = 'de:"Benicio Penalva" AND texto:(política OR partido)'

qp = QueryParser("texto", schema=ix.schema)
q = qp.parse(busca)
print(q)

(de:"Benicio  Penalva " AND (texto:política OR texto:partido))


O método *search* recebe uma busca processada pelo método *parse* e retorna um objeto do tipo *Results*. Para que todos os resultados obtidos sejam incluídos no objeto gerado, deve-se especificar 'limit=None'. 

In [16]:
s = ix.searcher()

resultados = s.search(q, limit=None, terms=True)

print('Foram gerados scores para ', resultados.scored_length(), 'documento(s)')

if resultados.has_matched_terms():
    matched = [(campo, termo.decode('utf-8')) for campo, termo in resultados.matched_terms()]
    print('Termos com match:\n', matched)

hits=[]

for hit in resultados:
    hits.append(hit)

Foram gerados scores para  10 documento(s)
Termos com match:
 [('de', 'Penalva'), ('de', 'Benicio'), ('texto', 'política'), ('texto', 'partido'), ('de', '')]


In [17]:
#if resultados.has_exact_length():
#    print("Score gerado para", found, "de exatamente", len(results), "documentos.")
#else:
#    low = resultados.estimated_min_length()
#    high = resultados.estimated_length()
#    print("Score gerado para", found, "de", low, "a", high, "documentos.")

Das 33 cartas que tratavam de questões políticas, apenas 10 foram encontradas. Seguem os documentos:

In [18]:
recuperados = jeremoabo.loc[jeremoabo.id.isin([hit['id'] for hit in hits]), :]
recuperados

,id,de,para,data,texto,tema
717,355-BPF-13-08-1890-mod.txt,Benicio Penalva de Faria (Benicio Penalva).,"Cícero Dantas Martins, Barão de Jeremoabo.",1890-08-13,"Meu Prezado Primo compadre e amigo Sr. Barão\n\n Bahia 13 de Agosto de 1890\n\n Anteontem estive com Joãozinho , por tê-lo encontrado no comércio , e- perguntando-lhe por Vossa Excelência , disse-me estar incomodado de sua saúde , o- que muito sinto : desejando que ao receber esta , se ache vigoroso ao lado da Excelentíssima Baronesa , com quem receberá nossas felicitações . O que há de mais importante aqui é a grande questão de esgotos 10 que não lhe é mais estranho . O marechal Hermes está mal de saúde , ouvi dizer que os- médicos o privarão de falar e que de vez em quando apresenta-lhe uma- hemorragia de sangue . O < redator > 11 do pequeno jornal tem brilhado\n\n nessa grande questão de esgotos , ultimamente disse que se o governador não sabia governar , que descesse dessa cadeira e intregasse-a a outro , esta frase é relativa aos indigentes da seca . Temos eleição no dia 15 de setembro 12 como sabe , e aqui estou sempre a- suas ordens , deliberando Vossa Excelência como melhor entender . No dia 17 , está anunciado uma grande reunião para formar-se o partido nacional , o que me diz a respeito ? Aqui ou em qualquer parte que esteja contará sempre com o meu fraco serviço . Tem chovido bastante aqui . Abençõe sua afilhada e dê- suas ordens a este que com toda\n\n estima , consideração e respeito é\n\n De Vossa Excelência Primo compadre e amigo pelo Coração\n\n Benicio Penalva",política
744,371-BPF-25-12-1896-mod.txt,Benicio Penalva de Faria.,Benicio Penalva de Faria.,1896-12-25,"Prezado compadre e amigo Senhor Barão\n\n Bahia 25 de Dezembro de 1896 .\n\n Que com minha Excelentíssima Comadre e Thotonio tenha tido boas festas e que o novo ano lhes seja venturoso , - muito estimamos , aceitando com todos nossas visitas . Sua carta de 16 deste , em resposta as minhas de 18 e 25 do passado e de 5 e- 11 deste , me foi entregue no dia 22 - V. não se persuadia que o povo do Conselheiro recuava perante a força Federal , embora fosse ele em número- muito inferior ? Já desenganou-se ? O Conselheiro e os 64 fanáticos dele , estão- muito audazes , mas creio , que se derem um novo combate , como espera-se , ficarão aniquilados para sempre , por que a força 65 que para alí seguiu , [de]ve 66 constar de 500 praças , entre [li] nha 67 e polícia , e foi muito bem ar[ma]da 68 e municiada ; Creio que levarão 6 ou 8 metralhadoras e 2 canhões Brupe , e quando puserem estas armas de guerra a funcionar em [li] nha 69 de combate , é um arra [so] 70 por que os conselheiristas não [conhe]cendo 71 o perigo , virão morrer braço a braço , e aí a fuzilaria fará a maior carnificina . Sinto muito a morte sas crianças , mas do outro povo não , por que nenhum deles está [ali] 72 sem um fim de pervesidade . Quanto ao Senhor Salles Souza , não procuro interpelá-lo , por estar [....] 73 constantemente inconveniente e não\n\n desejo entrar com ele em certas averiguações . Por sua carta , vejo que o seu prejuízo nas Fazendas , é enorme , e nem podia deixar de ser , visto sua ausência , delas , a 14 anos , e ainda mais , com a- dificuldades de encontrar vaqueiros que tenhão interesse . Então não toma parte na eleição de 30 deste mês ? Acho-lhe razão , e melhor seria que abandonasse de vez a política , por que , a não ser uma oposição séria em todo o Estado e bem- arregimentada , não vale apenas : mas com que garantias ficão os seus amigos que tanto se sacrificarão , acarretando as maiores odeosidades , por contarem com o seu apoio e do Doutor José Gonsalves ? Ficão entregues a mercê das iras dos dominantes da época , não é assim ? O Doutor José Gonsalves , cada vez mais se afunda no abismo da inocência , e depois virá apresentar razões frívolas para defender-se . Quem foi o culpado de toda esta guerra inglória que temos sofrido , e continuaremos , talvez para sempre ? Forão os soldados do partido , ou os seus

In [19]:
total = []
for i in recuperados.tema:
    total.append('política' in i)
    
print('Documento recuperados que tratam de política:', sum(total))

r = sum(total)/33
print('Revocação:', r)

p = sum(total)/len(recuperados.tema)
print('Precisão:', p)

Documento recuperados que tratam de política: 10
Revocação: 0.30303030303030304
Precisão: 1.0


In [20]:
s.close() # fechamos a busca

Os 10 documentos encontrados realmente dizem respeito à política. Dessa forma, a **revocação** do nosso método de busca foi de 10/33 = 30,3%, e a **precisão**, de 100%. 

Em seguida, para fins de comparação, repetimos a mesma busca utilizando o *analyzer* anterior para o campo 'texto' com um filtro de *stemming* adicional. Isso corresponde ao uso do *LanguageAnalyzer*. 

Para que o novo *analyzer* fosse utilizado nas buscas, foi necessário reconstruir o índice. 

In [21]:
schema = Schema(id=ID(unique=True, stored=True), 
                de=TEXT(stored=True,
                        analyzer=RegexTokenizer(expression='[a-zA-Z0-9áàâãéèêíïóôõöúçñÁÀÂÃÉÈÍÏÓÔÕÖÚÇÑ]*')), 
                para=TEXT(stored=True,
                          analyzer=RegexTokenizer(expression='[a-zA-Z0-9áàâãéèêíïóôõöúçñÁÀÂÃÉÈÍÏÓÔÕÖÚÇÑ]*')), 
                data=DATETIME(stored=True), 
                texto=TEXT(stored=True,  
                           analyzer=LanguageAnalyzer(lang='pt')))

ix = create_in('index_cartas', schema)

writer = ix.writer()

for i in cartas.index:
    if pd.isna(cartas.de[i]):
        cartas.de[i] = ''
        
    if pd.isna(cartas.para[i]):
        cartas.para[i] = '' 
    
    if pd.isna(cartas.data[i]):
        writer.add_document(id=cartas.id[i], 
                            de=cartas.de[i], 
                            para=cartas.para[i], 
                            texto=cartas.texto[i])
    else:
        writer.add_document(id=cartas.id[i], 
                            de=cartas.de[i], 
                            para=cartas.para[i], 
                            data=cartas.data[i], 
                            texto=cartas.texto[i])
writer.commit()

Reindexados os documentos, realizamos a mesma busca:

In [22]:
busca = 'de:"Benicio Penalva" AND texto:(política OR partido)'

qp = QueryParser("texto", schema=ix.schema)
q = qp.parse(busca)

s = ix.searcher()

resultados = s.search(q, limit=None, terms=True)

print('Foram gerados scores para ', resultados.scored_length(), 'documento(s)')

if resultados.has_matched_terms():
    matched = [(campo, termo.decode('utf-8')) for campo, termo in resultados.matched_terms()]
    print('Termos com match:\n', matched)

hits=[]

for hit in resultados:
    hits.append(hit)

Foram gerados scores para  14 documento(s)
Termos com match:
 [('de', 'Penalva'), ('de', 'Benicio'), ('texto', 'part'), ('de', ''), ('texto', 'polít')]


Os documentos recuperados foram os seguintes:

In [23]:
recuperados = jeremoabo.loc[jeremoabo.id.isin([hit['id'] for hit in hits]), :]
recuperados

,id,de,para,data,texto,tema
717,355-BPF-13-08-1890-mod.txt,Benicio Penalva de Faria (Benicio Penalva).,"Cícero Dantas Martins, Barão de Jeremoabo.",1890-08-13,"Meu Prezado Primo compadre e amigo Sr. Barão\n\n Bahia 13 de Agosto de 1890\n\n Anteontem estive com Joãozinho , por tê-lo encontrado no comércio , e- perguntando-lhe por Vossa Excelência , disse-me estar incomodado de sua saúde , o- que muito sinto : desejando que ao receber esta , se ache vigoroso ao lado da Excelentíssima Baronesa , com quem receberá nossas felicitações . O que há de mais importante aqui é a grande questão de esgotos 10 que não lhe é mais estranho . O marechal Hermes está mal de saúde , ouvi dizer que os- médicos o privarão de falar e que de vez em quando apresenta-lhe uma- hemorragia de sangue . O < redator > 11 do pequeno jornal tem brilhado\n\n nessa grande questão de esgotos , ultimamente disse que se o governador não sabia governar , que descesse dessa cadeira e intregasse-a a outro , esta frase é relativa aos indigentes da seca . Temos eleição no dia 15 de setembro 12 como sabe , e aqui estou sempre a- suas ordens , deliberando Vossa Excelência como melhor entender . No dia 17 , está anunciado uma grande reunião para formar-se o partido nacional , o que me diz a respeito ? Aqui ou em qualquer parte que esteja contará sempre com o meu fraco serviço . Tem chovido bastante aqui . Abençõe sua afilhada e dê- suas ordens a este que com toda\n\n estima , consideração e respeito é\n\n De Vossa Excelência Primo compadre e amigo pelo Coração\n\n Benicio Penalva",política
735,368-BPF-31-01-1895-mod.txt,Benicio Penalva de Faria.,"Cícero Dantas Martins, Barão de Jeremoabo.",1895-01-31,"Prezado Primo compadre e amigo Sr. Barão\n\n Bahia 31 de Janeiro de 1895 .\n\n Recebi sua carta de 23 deste , e muito estimo que com a Excelentíssima minha Comadre e seus caros filhos continue a gozar perfeita saúde e todos os bens da vida , recebendo com todos nossas sinceras visitas e saudades . ontem o Governador baixou o ato suspendendo a lei da organização da guarda municipal ; mas creio , que o Conselho e o Intendente , não se submeteu a semelhante arbitrariedade ; tanto assim , que consta-me ter feito hoje as- nomeações da dita guarda ou parte dela . O Senhor Governador , está- parecendo um governador de bob[ão] Consta que no Rio tem havi[do] 50 grande movimento dos E[studan]tes 51 da Escola militar e os Jacobi[nos] 52 com a Polícia e que alguns oficiais ou Generais tem apresentado-se em favor dos estudantes ; e por telegrama que vi no Jornal de Notícias , creio que de ontem ou anteontem , diz , estar a Armada de prontidão , pelo que , parece-me 53 que as cousas lá não vão boas . Parece-me que aqui haverá barulho antes do fim de Março , se o governador entender de fazer valer sua arbitrariedade . ontem recebi um telegrama de Barretto , dizendo-me que fizesse o filho , o estudante da Escola , seguir no primeiro vapor , e como ele [ai]nda 54 não chegou aqui , por isso [peço]-lhe 55 o favor de remeter-lhe a inclusa carta com toda brevidade , a fim dele não demorar-se mais- em casa dos Avós , visto a exigência do Pai . Sem dúvida alí no Rio é necessário a presença dele com brevidade . Já tirei minha patente e mandei ontem para a Comarca , acompanhada de uma procuração a fim de prestar o juramento e tomar posse . Recebi esta semana uma carta de meu filho , o que está no Rio , dizendo-me que consta estar o colera em Minas e já a um mês ou mais [...] 56 , e que aí no Rio também dizião ter aparecido alguns casos , mas que ele não acreditava , visto já estar com tanto tempo e a comunicação que tem aquele Estado-\n\n com o do Rio , todos os dias pelo trem de ferro , e não estar tudo devastado ; pela moléstia , e que o estado sanitário do Rio era presentemente o melhor possível ; o que faz-me também crer , que não é o cólera morbus que alí , como em Minas está graçando ; por que esta moléstia desenvolve-se com uma rapidez extraordinária , salvo porém , se agora ela vem mais moder

In [24]:
total = []
for i in recuperados.tema:
    total.append('política' in i)
    
print('Documento recuperados que tratam de política:', sum(total))

r = sum(total)/33
print('Revocação:', r)

p = sum(total)/len(recuperados.tema)
print('Precisão:', p)

Documento recuperados que tratam de política: 14
Revocação: 0.42424242424242425
Precisão: 1.0


In [25]:
s.close() # fechamos a busca

Os 14 documentos encontrados realmente dizem respeito à política. Assim, o uso do filtro de *stemming* melhorou a **revocação** do método de busca, que aumentou para 14/33 = 42,4%. A **precisão** continuou em 100%.

### Buscas expandidas (Questão 2 da Lista 1):

Para tentarmos melhorar os resultados gerados com o mecanismo de busca definido acima, criamos listas com termos equivalentes relacionados a temas associados à política. As listas foram criadas com dicionários de sinônimos disponíveis na internet (https://www.dicionarioinformal.com.br/sinonimos/, https://www.sinonimos.com.br/, https://synonyms.reverso.net/dicionario-sinonimos/pt/) variações de tempos verbais e pessoa para verbos.

In [26]:
politica = ['abordagem', 'administração', 'aproximação', 'artimanha', 'astúcia', 'atenção', 'atuação',
            'ação', 'caminho', 'cavilação', 'civilidade', 'conduta', 'condução', 'conselho', 'consenso', 
            'cortesia', 'delicadeza', 'diabrura', 'diretiva', 'diretriz', 'direção', 'dolo', 'enfoque', 
            'engenho', 'esperteza', 'estratagema', 'estratégia', 'gauchada', 'governação', 
            'governo', 'indicação', 'iniciativa', 'instrução', 'ladinice', 'lábia', 'maciota', 
            'malícia', 'manha', 'maquiavelice', 'maquiavelismo', 'maranha', 'norma', 'organização', 
            'orientação', 'regime', 'ronha', 'sagacidade', 'sentido', 'solércia', 'sutileza', 
            'tática', 'traquinada', 'travessura', 'via']

In [27]:
partido = ['agremiação', 'associação', 'bando', 'divisão', 'facção', 'grupo', 'liga', 'parcela', 'partidário',
           'representação', 'seita', 'sociedade', 'tribo', 'união']

A nova busca é representada pela seguinte cadeia de caracteres:

In [28]:
politica.extend(partido)
sinonimos = set(politica)

busca = 'de:"Benicio Penalva" AND texto:(política OR partido'

for i in sinonimos:
    busca += ' OR ' + i
busca += ')'

print(busca)

de:"Benicio Penalva" AND texto:(política OR partido OR atenção OR consenso OR estratagema OR grupo OR governo OR sutileza OR estratégia OR cavilação OR conduta OR delicadeza OR regime OR maquiavelismo OR aproximação OR bando OR astúcia OR cortesia OR maquiavelice OR traquinada OR associação OR direção OR facção OR gauchada OR governação OR representação OR diretriz OR iniciativa OR parcela OR civilidade OR diretiva OR maranha OR ronha OR norma OR ladinice OR liga OR via OR caminho OR instrução OR união OR orientação OR manha OR tática OR administração OR organização OR seita OR ação OR sentido OR condução OR lábia OR partidário OR sagacidade OR sociedade OR atuação OR divisão OR tribo OR malícia OR artimanha OR indicação OR maciota OR agremiação OR esperteza OR abordagem OR dolo OR enfoque OR travessura OR solércia OR diabrura OR conselho OR engenho)


Utilizando a nova busca, repetimos o procedimento anterior, utilizando novamente o *LanguageAnalyzer*.

In [29]:
qp = QueryParser("texto", schema=ix.schema)
q = qp.parse(busca)

s = ix.searcher()

resultados = s.search(q, limit=None, terms=True)

print('Foram gerados scores para ', resultados.scored_length(), 'documento(s)')

if resultados.has_matched_terms():
    matched = [(campo, termo.decode('utf-8')) for campo, termo in resultados.matched_terms()]
    print('Termos com match:\n', matched)

hits=[]

for hit in resultados:
    hits.append(hit)

Foram gerados scores para  36 documento(s)
Termos com match:
 [('texto', 'caminh'), ('de', 'Penalva'), ('de', 'Benicio'), ('texto', 'band'), ('texto', 'indic'), ('texto', 'conselh'), ('texto', 'lig'), ('texto', 'govern'), ('texto', 'part'), ('texto', 'sent'), ('texto', 'engenh'), ('texto', 'civil'), ('texto', 'atençã'), ('texto', 'administr'), ('de', ''), ('texto', 'polít'), ('texto', 'organiz'), ('texto', 'uniã')]


Os 36 documentos obtidos foram os seguintes:

In [30]:
recuperados = jeremoabo.loc[jeremoabo.id.isin([hit['id'] for hit in hits]), :]
recuperados

,id,de,para,data,texto,tema
716,354-BPF-27-03-1890-mod.txt,Benicio Penalva de Faria (Benicio Penalva).,"Cícero Dantas Martins, Barão de Jeremoabo.",1890-03-27,"Meu caro Primo compadre e amigo Sr. Barão .\n\n Bahia 27 de Março de 1890\n\n Aqui cheguei com minha família no dia 15 deste e só no dia 25 - pude ir a casa do Doutor Salustio , e não o encontrando , tornei hoje , e com ele estive conversando : apresentei-lhe a carta de Vossa Excelência como me ordenou . Disse-me o Doutor Salustio , que soube estarem esperando hoje pelo Visconde 5 do- Guahy , e que não sabia se era exata essa notícia : preveni ao- Doutor Salustio para falar ao Guahy , sobre um emprego no novo hospital da misericórdia , segundo Vossa Excelência me ter dito que queria ver se podia obter com o Guahy um bom emprego para mim-\n\n no dito hospital . Vossa Excelência sabe que muito e muito preciso , e que só me arrisquei a vir com a família para esta capital , com único fim de dar a meus filhos uma educação mais ou menos regular , - pois é o meu maior desideratum . Já matriculei dois meninos- no Lyceo , e minha filha hoje , foi 6 também matriculada no externato normal de Senhoras . Se eu puder conseguir os meus- desejos , é um meio de vida que dou a ela , visto não termos outro de que possa ela viver . Sua afilhada , e o do Doutor Severino , ainda não pôde levá-los a escola , por faltar- me um\n\n atestado médico , de como não sofrem moléstia alguma contagiosa , do 7 que só poderei dar andamento depois da semana Santa . Tenho andado tanto aqui , que já 8 me doem as pernas , de maneiras que , para caminhar , é em marcha muito lenta . Tenho feito muita despesa com as arrumações dos meninos , bem entendido , de conformidade a minhas forças , por que cada um vive como pode , e- como sabe , não tenho recursos . Provisoriamente estou morando- na rua do Alvo , nº 201 – 2º andar , em quanto procuro uma roça para ir tendo alguma cousa com que possa ir dando a sub-\n\n a subsistência a meus filhos : Deus é grande e não desampara- a ninguém , principalmente a quem faz diligência com fé nele . Logo que ache a roça e arrende , preciso comprar uns burros 9 , e- desde já lhe previno , por serem os de Vossa Excelência reforçados . Meu filho Francisco , desde que aqui- chegou , entrou em tratamento com o Doutor Ribeiro dos Santos , por se achar sofrendo bastante dos olhos . Aqui tem chovido alguma cousa , e consta-me que por lá também ; Deus permita que continue . Sua Comadre está muito satisfeita aqui , - por ter o mesmo desejo que eu\n\n tenho de ver os filhos educados . Não temos tido alteração em nossa saúde graças a Deus . Muito estimo que ao lado da Excelentíssima Baronesa frua perfeita saúde e todos os- bens . Abençõe sua afilhada , e disponha sempre com a maior- franqueza , do pouco préstimo deste que com muita estima e- melhor consideração é\n\n De Vossa Excelência Primo compadre e amigo pelo Coração\n\n Benicio Penalva . P. S. Recebeu uma carta com a- escritura para o Sr. Boaventura ? Remetia pelo correio do Timbó- que devia seguir no dia 17 deste .\n\n O mesmo","emprego, educação, saúde"
717,355-BPF-13-08-1890-mod.txt,Benicio Penalva de Faria (Benicio Penalva).,"Cícero Dantas Martins, Barão de Jeremoabo.",1890-08-13,"Meu Prezado Primo compadre e amigo Sr. Barão\n\n Bahia 13 de Agosto de 1890\n\n Anteontem estive com Joãozinho , por tê-lo encontrado no comércio , e- perguntando-lhe por Vossa Excelência , disse-me estar incomodado de sua saúde , o- que muito sinto : desejando que ao receber esta , se ache vigoroso ao lado da Excelentíssima Baronesa , com quem receberá nossas felicitações . O que há de mais importante aqui é a grande questão de esgotos 10 que não lhe é mais estranho . O marechal Hermes está mal de saúde , ouvi dizer que os- médicos o privarão de falar e que de vez em quando apresenta-lhe uma- hemorragia de sangue . O < redator > 11 do pequeno jornal tem brilhado\n\n nessa grande questão de esgotos , ultimamente disse que se o governador não sabia gover

In [31]:
total = []
for i in recuperados.tema:
    total.append('política' in i)
    
print('Documento recuperados que tratam de política:', sum(total))

r = sum(total)/33
print('Revocação:', r)

p = sum(total)/len(recuperados.tema)
print('Precisão:', p)

Documento recuperados que tratam de política: 28
Revocação: 0.8484848484848485
Precisão: 0.7777777777777778


In [32]:
s.close() # fechamos a busca

Vinte e oito dos documentos encontrados tratam de política. Assim, o uso dos termos equivalentes aumentou consideravelmente a **revocação** do método de busca, que para 84,8%. Em contrapartida, a **precisão** foi reduzida para 77,8%.

### Correção ortográfica (Questão 3 da Lista 1):

Até agora, assumimos que os termos usados nas buscas estavam corretos sob o ponto de vista ortográfico. Na prática, isso, muitas vezes, não ocorre. Para lidar com essa situação, podemos recorrer a um corretor ortográfico, que identificaria na lista de termos os que não estivessem corretos, sugerindo correções.

Vamos implementar essa funcionalidade antes que a cadeia de caracteres seja passada para o método *parse* do Whoosh. Na realidade, após a correção, deveríamos identificar termos equivalentes e só então passá-los para o *parse*. Os resultados obtidos na análise a seguir devem ser comparados apenas com os de processos de busca similares, ou seja, que não utilizem listas de termos equivalentes.

A biblioteca de correção ortográfica utilizada foi a [pyspellchecker](https://pyspellchecker.readthedocs.io/en/latest/). Uma das línguas implementadas na biblioteca é o português. 

In [33]:
spell = SpellChecker(language='pt')

termos = ['poltica', 'plitica', 'politca', 'paritdos', 'prtido', 'plítca', 'pratidos']
corretos = [spell.correction(termo) for termo in termos]

print(corretos)

['política', 'politica', 'politica', 'partidos', 'partido', 'política', 'partidos']


A biblioteca retorna o termo correto para uma boa parte dos erros cometidos na lista acima. 

In [34]:
busca = 'de:"Benicio Penalva" AND texto:(política OR partido'

for i in corretos:
    busca += ' OR ' + i
busca += ')'

print(busca)

de:"Benicio Penalva" AND texto:(política OR partido OR política OR politica OR politica OR partidos OR partido OR política OR partidos)


Como os termos corrigidos foram praticamente iguais aos da busca original, os documentos recuperados foram os mesmos.

In [35]:
qp = QueryParser("texto", schema=ix.schema)
q = qp.parse(busca)

s = ix.searcher()

resultados = s.search(q, limit=None, terms=True)

print('Foram gerados scores para ', resultados.scored_length(), 'documento(s)')

if resultados.has_matched_terms():
    matched = [(campo, termo.decode('utf-8')) for campo, termo in resultados.matched_terms()]
    print('Termos com match:\n', matched)

hits=[]

for hit in resultados:
    hits.append(hit)

Foram gerados scores para  14 documento(s)
Termos com match:
 [('de', 'Penalva'), ('de', 'Benicio'), ('texto', 'polit'), ('texto', 'part'), ('de', ''), ('texto', 'polít')]


Os 14 documentos obtidos foram os seguintes:

In [36]:
recuperados = jeremoabo.loc[jeremoabo.id.isin([hit['id'] for hit in hits]), :]
recuperados

,id,de,para,data,texto,tema
717,355-BPF-13-08-1890-mod.txt,Benicio Penalva de Faria (Benicio Penalva).,"Cícero Dantas Martins, Barão de Jeremoabo.",1890-08-13,"Meu Prezado Primo compadre e amigo Sr. Barão\n\n Bahia 13 de Agosto de 1890\n\n Anteontem estive com Joãozinho , por tê-lo encontrado no comércio , e- perguntando-lhe por Vossa Excelência , disse-me estar incomodado de sua saúde , o- que muito sinto : desejando que ao receber esta , se ache vigoroso ao lado da Excelentíssima Baronesa , com quem receberá nossas felicitações . O que há de mais importante aqui é a grande questão de esgotos 10 que não lhe é mais estranho . O marechal Hermes está mal de saúde , ouvi dizer que os- médicos o privarão de falar e que de vez em quando apresenta-lhe uma- hemorragia de sangue . O < redator > 11 do pequeno jornal tem brilhado\n\n nessa grande questão de esgotos , ultimamente disse que se o governador não sabia governar , que descesse dessa cadeira e intregasse-a a outro , esta frase é relativa aos indigentes da seca . Temos eleição no dia 15 de setembro 12 como sabe , e aqui estou sempre a- suas ordens , deliberando Vossa Excelência como melhor entender . No dia 17 , está anunciado uma grande reunião para formar-se o partido nacional , o que me diz a respeito ? Aqui ou em qualquer parte que esteja contará sempre com o meu fraco serviço . Tem chovido bastante aqui . Abençõe sua afilhada e dê- suas ordens a este que com toda\n\n estima , consideração e respeito é\n\n De Vossa Excelência Primo compadre e amigo pelo Coração\n\n Benicio Penalva",política
735,368-BPF-31-01-1895-mod.txt,Benicio Penalva de Faria.,"Cícero Dantas Martins, Barão de Jeremoabo.",1895-01-31,"Prezado Primo compadre e amigo Sr. Barão\n\n Bahia 31 de Janeiro de 1895 .\n\n Recebi sua carta de 23 deste , e muito estimo que com a Excelentíssima minha Comadre e seus caros filhos continue a gozar perfeita saúde e todos os bens da vida , recebendo com todos nossas sinceras visitas e saudades . ontem o Governador baixou o ato suspendendo a lei da organização da guarda municipal ; mas creio , que o Conselho e o Intendente , não se submeteu a semelhante arbitrariedade ; tanto assim , que consta-me ter feito hoje as- nomeações da dita guarda ou parte dela . O Senhor Governador , está- parecendo um governador de bob[ão] Consta que no Rio tem havi[do] 50 grande movimento dos E[studan]tes 51 da Escola militar e os Jacobi[nos] 52 com a Polícia e que alguns oficiais ou Generais tem apresentado-se em favor dos estudantes ; e por telegrama que vi no Jornal de Notícias , creio que de ontem ou anteontem , diz , estar a Armada de prontidão , pelo que , parece-me 53 que as cousas lá não vão boas . Parece-me que aqui haverá barulho antes do fim de Março , se o governador entender de fazer valer sua arbitrariedade . ontem recebi um telegrama de Barretto , dizendo-me que fizesse o filho , o estudante da Escola , seguir no primeiro vapor , e como ele [ai]nda 54 não chegou aqui , por isso [peço]-lhe 55 o favor de remeter-lhe a inclusa carta com toda brevidade , a fim dele não demorar-se mais- em casa dos Avós , visto a exigência do Pai . Sem dúvida alí no Rio é necessário a presença dele com brevidade . Já tirei minha patente e mandei ontem para a Comarca , acompanhada de uma procuração a fim de prestar o juramento e tomar posse . Recebi esta semana uma carta de meu filho , o que está no Rio , dizendo-me que consta estar o colera em Minas e já a um mês ou mais [...] 56 , e que aí no Rio também dizião ter aparecido alguns casos , mas que ele não acreditava , visto já estar com tanto tempo e a comunicação que tem aquele Estado-\n\n com o do Rio , todos os dias pelo trem de ferro , e não estar tudo devastado ; pela moléstia , e que o estado sanitário do Rio era presentemente o melhor possível ; o que faz-me também crer , que não é o cólera morbus que alí , como em Minas está graçando ; por que esta moléstia desenvolve-se com uma rapidez extraordinária , salvo porém , se agora ela vem mais moder

In [37]:
total = []
for i in recuperados.tema:
    total.append('política' in i)
    
print('Documento recuperados que tratam de política:', sum(total))

r = sum(total)/33
print('Revocação:', r)

p = sum(total)/len(recuperados.tema)
print('Precisão:', p)

Documento recuperados que tratam de política: 14
Revocação: 0.42424242424242425
Precisão: 1.0


In [38]:
s.close() # fechamos a busca

Os 14 documentos encontrados foram os mesmos obtidos com a busca realizada com o *LanguageAnalyzer* sem o uso de termos equivalentes. A **revocação** (42,4%) e a **precisão** (100%) do método de busca foram as mesmas.

### Consulta por frases (Questão 4 da Lista 1):

Quase 10% das buscas realizadas no Google têm por base uma frase, em vez de palavras "soltas". A consulta por frases é implementada no Whoosh no seu *parser* padrão (https://whoosh.readthedocs.io/en/latest/parsing.html#allowing-complex-phrase-queries). Uma frase é identificada pelo *parser* quando a busca consiste em uma sequência de termos entre aspas. O *parser* tokeniza e filtra os termos da frase e gera uma consulta por proximidade. É possível também especificar o número máximo de termos permitido entre os termos da frase processada pelo *parser*.

Para ilustrar sua aplicação, vamos utilizar as 11 cartas de Benício Penalva em que ele solicita a ajuda do Barão de Jeremoabo para conseguir um emprego para si mesmo ou para pessoas próximas. 

In [39]:
lista = []

for i in jeremoabo.tema.str.find('emprego'):
    if i == -1:
        lista.append(False)
    else:
        lista.append(True)
        
emprego_jeremoabo = jeremoabo.loc[lista, :]
emprego_jeremoabo

,id,de,para,data,texto,tema
716,354-BPF-27-03-1890-mod.txt,Benicio Penalva de Faria (Benicio Penalva).,"Cícero Dantas Martins, Barão de Jeremoabo.",1890-03-27,"Meu caro Primo compadre e amigo Sr. Barão .\n\n Bahia 27 de Março de 1890\n\n Aqui cheguei com minha família no dia 15 deste e só no dia 25 - pude ir a casa do Doutor Salustio , e não o encontrando , tornei hoje , e com ele estive conversando : apresentei-lhe a carta de Vossa Excelência como me ordenou . Disse-me o Doutor Salustio , que soube estarem esperando hoje pelo Visconde 5 do- Guahy , e que não sabia se era exata essa notícia : preveni ao- Doutor Salustio para falar ao Guahy , sobre um emprego no novo hospital da misericórdia , segundo Vossa Excelência me ter dito que queria ver se podia obter com o Guahy um bom emprego para mim-\n\n no dito hospital . Vossa Excelência sabe que muito e muito preciso , e que só me arrisquei a vir com a família para esta capital , com único fim de dar a meus filhos uma educação mais ou menos regular , - pois é o meu maior desideratum . Já matriculei dois meninos- no Lyceo , e minha filha hoje , foi 6 também matriculada no externato normal de Senhoras . Se eu puder conseguir os meus- desejos , é um meio de vida que dou a ela , visto não termos outro de que possa ela viver . Sua afilhada , e o do Doutor Severino , ainda não pôde levá-los a escola , por faltar- me um\n\n atestado médico , de como não sofrem moléstia alguma contagiosa , do 7 que só poderei dar andamento depois da semana Santa . Tenho andado tanto aqui , que já 8 me doem as pernas , de maneiras que , para caminhar , é em marcha muito lenta . Tenho feito muita despesa com as arrumações dos meninos , bem entendido , de conformidade a minhas forças , por que cada um vive como pode , e- como sabe , não tenho recursos . Provisoriamente estou morando- na rua do Alvo , nº 201 – 2º andar , em quanto procuro uma roça para ir tendo alguma cousa com que possa ir dando a sub-\n\n a subsistência a meus filhos : Deus é grande e não desampara- a ninguém , principalmente a quem faz diligência com fé nele . Logo que ache a roça e arrende , preciso comprar uns burros 9 , e- desde já lhe previno , por serem os de Vossa Excelência reforçados . Meu filho Francisco , desde que aqui- chegou , entrou em tratamento com o Doutor Ribeiro dos Santos , por se achar sofrendo bastante dos olhos . Aqui tem chovido alguma cousa , e consta-me que por lá também ; Deus permita que continue . Sua Comadre está muito satisfeita aqui , - por ter o mesmo desejo que eu\n\n tenho de ver os filhos educados . Não temos tido alteração em nossa saúde graças a Deus . Muito estimo que ao lado da Excelentíssima Baronesa frua perfeita saúde e todos os- bens . Abençõe sua afilhada , e disponha sempre com a maior- franqueza , do pouco préstimo deste que com muita estima e- melhor consideração é\n\n De Vossa Excelência Primo compadre e amigo pelo Coração\n\n Benicio Penalva . P. S. Recebeu uma carta com a- escritura para o Sr. Boaventura ? Remetia pelo correio do Timbó- que devia seguir no dia 17 deste .\n\n O mesmo","emprego, educação, saúde"
718,356-BPF-19-09-1890-mod.txt,Benicio Penalva de Faria (Benicio Penalva).,"Cícero Dantas Martins, Barão de Jeremoabo.",1890-09-19,"Meu Prezado Primo compadre e amigo Sr. Barão .\n\n Bahia 19 de Setembro 13 de 1890\n\n Desejo que com a Excelentíssima Baronesa frua perfeita saúde e todos o bens . Hoje encontrei-me no Comércio- com Joãozinho , e perguntando por Totonho , disse-me estar bom . Até hoje está o seu nome incluído na lista- dos candidatos mais votados , e terei grande prazer se for eleito . Como sei que Vossa Excelência é amigo do Conselheiro Virgilio Damasio , e está ele na administração do estado , peço-lhe , de sendo-lhe possível , obter com ele o lugar de inspetor da linha telegráfica do governo , no Estado de gergipe [*] 14 , numa seção que está vaga\n\n e adida ao inspetor Antonio Ribeiro , ou outro lugar aqui na Capital , que Vossa Excelência julgar conveniente . O D

Inicialmente, realizaremos a busca pela frase exata "obter um lugar", que aparece em apenas um dos documentos acima listados.

In [40]:
busca = 'de:"Benicio Penalva" AND texto:"obter um lugar"~1'

qp = QueryParser("texto", schema=ix.schema)
q = qp.parse(busca)
print(q)

(de:"Benicio  Penalva " AND texto:"obter lug")


Pode-se verificar que o *parser* tokenizou e removeu as *stopwords* da frase a ser buscada. Como nas buscas anteriores, passamos a *query* resultante para o método *search* do objeto *searcher* que criamos.

In [41]:
s = ix.searcher()

resultados = s.search(q, limit=None, terms=True)

print('Foram gerados scores para ', resultados.scored_length(), 'documento(s)')

if resultados.has_matched_terms():
    matched = [(campo, termo.decode('utf-8')) for campo, termo in resultados.matched_terms()]
    print('Termos com match:\n', matched)

hits=[]

for hit in resultados:
    hits.append(hit)

Foram gerados scores para  2 documento(s)
Termos com match:
 [('de', 'Penalva'), ('de', 'Benicio'), ('texto', 'lug'), ('texto', 'obter'), ('de', '')]


Apesar de apenas uma das 11 cartas que tratavam de solicitações de emprego conter a frase exata, dois documentos foram recuperados. Isso ocorreu porque os documentos também haviam sido processados com o *LanguageAnalyzer*, que remove as *stopwords*. Os documentos obtidos foram:

  * 356-BPF-19-09-1890-mod.txt: "obter com ele o lugar de inspetor"
  * 360-BPF-23-10-1890-mod.txt: "obter um lugar para mim", "obter o lugar de administrador"

In [42]:
recuperados = jeremoabo.loc[jeremoabo.id.isin([hit['id'] for hit in hits]), :]
recuperados

,id,de,para,data,texto,tema
718,356-BPF-19-09-1890-mod.txt,Benicio Penalva de Faria (Benicio Penalva).,"Cícero Dantas Martins, Barão de Jeremoabo.",1890-09-19,"Meu Prezado Primo compadre e amigo Sr. Barão .\n\n Bahia 19 de Setembro 13 de 1890\n\n Desejo que com a Excelentíssima Baronesa frua perfeita saúde e todos o bens . Hoje encontrei-me no Comércio- com Joãozinho , e perguntando por Totonho , disse-me estar bom . Até hoje está o seu nome incluído na lista- dos candidatos mais votados , e terei grande prazer se for eleito . Como sei que Vossa Excelência é amigo do Conselheiro Virgilio Damasio , e está ele na administração do estado , peço-lhe , de sendo-lhe possível , obter com ele o lugar de inspetor da linha telegráfica do governo , no Estado de gergipe [*] 14 , numa seção que está vaga\n\n e adida ao inspetor Antonio Ribeiro , ou outro lugar aqui na Capital , que Vossa Excelência julgar conveniente . O Doutor Loizio , continua- na administração de engenheiro- geral dos telégrafos , pelo que ju julgo que não lhe será muito difícil conseguir para mim o lugar de inspetor , como peço . Vossa Excelência não ignora que muito preciso , por ser este uma meio fácil de conseguir a educação de meus filhos . Fiquei admirado não ter Vossa Excelência um só voto no Palácio [...] 15 ( termo do Conde ) ao passo que o Doutor Severino teve no Barração 180 - Como se explica isto ? ! Abençõe- sua afilhada , e sempre a seu dispor encontrará o pouco préstimo deste que é com toda estima e consideração\n\n De Vossa Excelência Primo compadre e amigo pelo Coração\n\n Benicio Penalva","emprego, política"
727,360-BPF-23-10-1890-mod.txt,Benicio Penalva de Faria (Benicio Penalva).,"Cícero Dantas Martins, Barão de Jeremoabo.",1890-10-23,"Meu Prezado Primo compadre e amigo\n\n Timbó 23 de outubro 19 de 1890\n\n É meu prazer que ao lado da Excelentíssima Baronesa e caros filhos frua sempre vigorosa saúde e todos os bens . Está na administração deste estado o seu distinto amigo o- Doutor José Gonçalvez , pelo que , em vista de minha necessidade , peço-lhe de com ele obter um lugar para mim na alfândega ou tesouraria geral , que não dependa de acesso ou concurso . Seria para mim uma felicidade- se pudesse obter o lugar de administrador das Capatorias da Alfândega , e se não for possível\n\n aceitarei outro , em quanto se arranja cousa melhor . O lugar de inspetor da linha telegráfica- no estado de Sergipe , da seção- vaga , que está adida ao Antônio Ribeiro , também me serve : finalmente , Vossa Excelência fará o que melhor entender a meu benefício ; com- tanto que seja empregado numa repartição da fazenda nacional , que me dê quanto chegue para minha subsistência e de minha família . Pode responder-me para Alagoinhas ou Timbó . Aguardo sua resposta e com a maior franqueza disponha sempre do pouco préstimo deste que é com toda estima e consideração\n\n De Vossa Excelência Primo amigo obrigadíssimo pelo Coração\n\n Benicio Penalva",emprego


In [43]:
total = []
for i in recuperados.tema:
    total.append('política' in i)
    
print('Documento recuperados que tratam de política:', sum(total))

r = sum(total)/33
print('Revocação:', r)

p = sum(total)/len(recuperados.tema)
print('Precisão:', p)

Documento recuperados que tratam de política: 1
Revocação: 0.030303030303030304
Precisão: 0.5


In [44]:
s.close() # fechamos a busca

Apenas uma das cartas de Benício Penalva contem a frase "obter com o Guahy um bom emprego". Nela, os termos "obter" e "emprego" aparecem separados por cinco termos. Destes, dois não são *stopwords*: "Guahy" e "bom".

In [45]:
stopper = StopFilter(lang='pt')
stemmer = StemFilter(lang='pt')

termos = stopper(RexTokenizer(u"obter com o Guahy um bom emprego"))

for termo in stemmer(termos):
    print(termo.text)

obter
guahy
bom
empreg


Como esperado, quando realizamos a busca pela frase exata, não obtemos resultados.

In [46]:
busca = 'de:"Benicio Penalva" AND texto:"obter um emprego"~1'

qp = QueryParser("texto", schema=ix.schema)
q = qp.parse(busca)
print(q)

(de:"Benicio  Penalva " AND texto:"obter empreg")


In [47]:
s = ix.searcher()

resultados = s.search(q, limit=None, terms=True)

print('Foram gerados scores para ', resultados.scored_length(), 'documento(s)')

Foram gerados scores para  0 documento(s)


Quando realizamos a mesma busca com um *slop* (número máximo de termos até o próximo termo da frase) de três termos, recuperamos o documento corretamente.

In [48]:
busca = 'de:"Benicio Penalva" AND texto:"obter um emprego"~3'

qp = QueryParser("texto", schema=ix.schema)
q = qp.parse(busca)
print(q)

(de:"Benicio  Penalva " AND texto:"obter empreg")


In [49]:
s = ix.searcher()

resultados = s.search(q, limit=None, terms=True)

print('Foram gerados scores para ', resultados.scored_length(), 'documento(s)')

if resultados.has_matched_terms():
    matched = [(campo, termo.decode('utf-8')) for campo, termo in resultados.matched_terms()]
    print('Termos com match:\n', matched)

hits=[]

for hit in resultados:
    hits.append(hit)

Foram gerados scores para  1 documento(s)
Termos com match:
 [('de', 'Penalva'), ('de', 'Benicio'), ('texto', 'empreg'), ('texto', 'obter'), ('de', '')]


In [50]:
recuperados = jeremoabo.loc[jeremoabo.id.isin([hit['id'] for hit in hits]), :]
recuperados

,id,de,para,data,texto,tema
716,354-BPF-27-03-1890-mod.txt,Benicio Penalva de Faria (Benicio Penalva).,"Cícero Dantas Martins, Barão de Jeremoabo.",1890-03-27,"Meu caro Primo compadre e amigo Sr. Barão .\n\n Bahia 27 de Março de 1890\n\n Aqui cheguei com minha família no dia 15 deste e só no dia 25 - pude ir a casa do Doutor Salustio , e não o encontrando , tornei hoje , e com ele estive conversando : apresentei-lhe a carta de Vossa Excelência como me ordenou . Disse-me o Doutor Salustio , que soube estarem esperando hoje pelo Visconde 5 do- Guahy , e que não sabia se era exata essa notícia : preveni ao- Doutor Salustio para falar ao Guahy , sobre um emprego no novo hospital da misericórdia , segundo Vossa Excelência me ter dito que queria ver se podia obter com o Guahy um bom emprego para mim-\n\n no dito hospital . Vossa Excelência sabe que muito e muito preciso , e que só me arrisquei a vir com a família para esta capital , com único fim de dar a meus filhos uma educação mais ou menos regular , - pois é o meu maior desideratum . Já matriculei dois meninos- no Lyceo , e minha filha hoje , foi 6 também matriculada no externato normal de Senhoras . Se eu puder conseguir os meus- desejos , é um meio de vida que dou a ela , visto não termos outro de que possa ela viver . Sua afilhada , e o do Doutor Severino , ainda não pôde levá-los a escola , por faltar- me um\n\n atestado médico , de como não sofrem moléstia alguma contagiosa , do 7 que só poderei dar andamento depois da semana Santa . Tenho andado tanto aqui , que já 8 me doem as pernas , de maneiras que , para caminhar , é em marcha muito lenta . Tenho feito muita despesa com as arrumações dos meninos , bem entendido , de conformidade a minhas forças , por que cada um vive como pode , e- como sabe , não tenho recursos . Provisoriamente estou morando- na rua do Alvo , nº 201 – 2º andar , em quanto procuro uma roça para ir tendo alguma cousa com que possa ir dando a sub-\n\n a subsistência a meus filhos : Deus é grande e não desampara- a ninguém , principalmente a quem faz diligência com fé nele . Logo que ache a roça e arrende , preciso comprar uns burros 9 , e- desde já lhe previno , por serem os de Vossa Excelência reforçados . Meu filho Francisco , desde que aqui- chegou , entrou em tratamento com o Doutor Ribeiro dos Santos , por se achar sofrendo bastante dos olhos . Aqui tem chovido alguma cousa , e consta-me que por lá também ; Deus permita que continue . Sua Comadre está muito satisfeita aqui , - por ter o mesmo desejo que eu\n\n tenho de ver os filhos educados . Não temos tido alteração em nossa saúde graças a Deus . Muito estimo que ao lado da Excelentíssima Baronesa frua perfeita saúde e todos os- bens . Abençõe sua afilhada , e disponha sempre com a maior- franqueza , do pouco préstimo deste que com muita estima e- melhor consideração é\n\n De Vossa Excelência Primo compadre e amigo pelo Coração\n\n Benicio Penalva . P. S. Recebeu uma carta com a- escritura para o Sr. Boaventura ? Remetia pelo correio do Timbó- que devia seguir no dia 17 deste .\n\n O mesmo","emprego, educação, saúde"


### Consulta híbrida (Questão 5 da Lista 1):

Em seguida, aperfeiçoamos a busca anterior de forma que, quando a consulta por uma frase exata não retornasse resultados, fosse feita uma busca booleana simples com os termos da frase. Como exemplo, usaremos a mesma busca pela frase "obter um emprego".

In [51]:
frase = '"obter um emprego"'
busca = 'de:"Benicio Penalva" AND texto:' + frase + '~1'

qp = QueryParser("texto", schema=ix.schema)
q = qp.parse(busca)
print(q)

(de:"Benicio  Penalva " AND texto:"obter empreg")


In [52]:
s = ix.searcher()

resultados = s.search(q, limit=None, terms=True)

if resultados.scored_length() > 0:
    print('Foram gerados scores para ', resultados.scored_length(), 'documento(s)')
else:
    frase = frase.replace(' ', ' OR ').replace('"', '')
    busca = 'de:"Benicio Penalva" AND texto:' + '(' + frase + ')'
    q = qp.parse(busca)
    resultados = s.search(q, limit=None, terms=True)
    print('Foram gerados scores para ', resultados.scored_length(), 'documento(s)')

if resultados.has_matched_terms():
    matched = [(campo, termo.decode('utf-8')) for campo, termo in resultados.matched_terms()]
    print('Termos com match:\n', matched)

hits=[]

for hit in resultados:
    hits.append(hit)

Foram gerados scores para  10 documento(s)
Termos com match:
 [('de', 'Benicio'), ('de', 'Penalva'), ('texto', 'empreg'), ('texto', 'obter'), ('de', '')]


In [53]:
recuperados = jeremoabo.loc[jeremoabo.id.isin([hit['id'] for hit in hits]), :]
recuperados

,id,de,para,data,texto,tema
716,354-BPF-27-03-1890-mod.txt,Benicio Penalva de Faria (Benicio Penalva).,"Cícero Dantas Martins, Barão de Jeremoabo.",1890-03-27,"Meu caro Primo compadre e amigo Sr. Barão .\n\n Bahia 27 de Março de 1890\n\n Aqui cheguei com minha família no dia 15 deste e só no dia 25 - pude ir a casa do Doutor Salustio , e não o encontrando , tornei hoje , e com ele estive conversando : apresentei-lhe a carta de Vossa Excelência como me ordenou . Disse-me o Doutor Salustio , que soube estarem esperando hoje pelo Visconde 5 do- Guahy , e que não sabia se era exata essa notícia : preveni ao- Doutor Salustio para falar ao Guahy , sobre um emprego no novo hospital da misericórdia , segundo Vossa Excelência me ter dito que queria ver se podia obter com o Guahy um bom emprego para mim-\n\n no dito hospital . Vossa Excelência sabe que muito e muito preciso , e que só me arrisquei a vir com a família para esta capital , com único fim de dar a meus filhos uma educação mais ou menos regular , - pois é o meu maior desideratum . Já matriculei dois meninos- no Lyceo , e minha filha hoje , foi 6 também matriculada no externato normal de Senhoras . Se eu puder conseguir os meus- desejos , é um meio de vida que dou a ela , visto não termos outro de que possa ela viver . Sua afilhada , e o do Doutor Severino , ainda não pôde levá-los a escola , por faltar- me um\n\n atestado médico , de como não sofrem moléstia alguma contagiosa , do 7 que só poderei dar andamento depois da semana Santa . Tenho andado tanto aqui , que já 8 me doem as pernas , de maneiras que , para caminhar , é em marcha muito lenta . Tenho feito muita despesa com as arrumações dos meninos , bem entendido , de conformidade a minhas forças , por que cada um vive como pode , e- como sabe , não tenho recursos . Provisoriamente estou morando- na rua do Alvo , nº 201 – 2º andar , em quanto procuro uma roça para ir tendo alguma cousa com que possa ir dando a sub-\n\n a subsistência a meus filhos : Deus é grande e não desampara- a ninguém , principalmente a quem faz diligência com fé nele . Logo que ache a roça e arrende , preciso comprar uns burros 9 , e- desde já lhe previno , por serem os de Vossa Excelência reforçados . Meu filho Francisco , desde que aqui- chegou , entrou em tratamento com o Doutor Ribeiro dos Santos , por se achar sofrendo bastante dos olhos . Aqui tem chovido alguma cousa , e consta-me que por lá também ; Deus permita que continue . Sua Comadre está muito satisfeita aqui , - por ter o mesmo desejo que eu\n\n tenho de ver os filhos educados . Não temos tido alteração em nossa saúde graças a Deus . Muito estimo que ao lado da Excelentíssima Baronesa frua perfeita saúde e todos os- bens . Abençõe sua afilhada , e disponha sempre com a maior- franqueza , do pouco préstimo deste que com muita estima e- melhor consideração é\n\n De Vossa Excelência Primo compadre e amigo pelo Coração\n\n Benicio Penalva . P. S. Recebeu uma carta com a- escritura para o Sr. Boaventura ? Remetia pelo correio do Timbó- que devia seguir no dia 17 deste .\n\n O mesmo","emprego, educação, saúde"
718,356-BPF-19-09-1890-mod.txt,Benicio Penalva de Faria (Benicio Penalva).,"Cícero Dantas Martins, Barão de Jeremoabo.",1890-09-19,"Meu Prezado Primo compadre e amigo Sr. Barão .\n\n Bahia 19 de Setembro 13 de 1890\n\n Desejo que com a Excelentíssima Baronesa frua perfeita saúde e todos o bens . Hoje encontrei-me no Comércio- com Joãozinho , e perguntando por Totonho , disse-me estar bom . Até hoje está o seu nome incluído na lista- dos candidatos mais votados , e terei grande prazer se for eleito . Como sei que Vossa Excelência é amigo do Conselheiro Virgilio Damasio , e está ele na administração do estado , peço-lhe , de sendo-lhe possível , obter com ele o lugar de inspetor da linha telegráfica do governo , no Estado de gergipe [*] 14 , numa seção que está vaga\n\n e adida ao inspetor Antonio Ribeiro , ou outro lugar aqui na Capital , que Vossa Excelência julgar conveniente . O D

Todos os documentos recuperados incluem variações dos termos "obter" ou "emprego" após a passagem pelo filtro de *stemming*. Por exemplo, no documento 396-BPF-21-09-1902-mod.txt, há uma ocorrência do termo "obterem".